## Cleaned up version of creating Visualizations

In [ ]:
model_names = ["Original-Pretrained-R2plus1DMotionSegNet_model.pth", "dropout-0_10-R2plus1DMotionSegNet_model.pth", "dropout-0_25-R2plus1DMotionSegNet_model.pth", "dropout-0_50-R2plus1DMotionSegNet_model.pth", "dropout-0_75-R2plus1DMotionSegNet_model.pth"]

In [ ]:
# change os curr dir to be one below such that src can be imported :/
import os
os.chdir("..")

In [ ]:
%config Completer.use_jedi = False

import echonet
from echonet.datasets import Echo

import torch.nn.functional as F
from torchvision.models.video import r2plus1d_18
from torch.utils.data import Dataset, DataLoader, Subset
from multiprocessing import cpu_count
from src.utils.torch_utils import TransformDataset, torch_collate
from src.transform_utils import generate_2dmotion_field
from src.loss_functions import huber_loss, convert_to_1hot, convert_to_1hot_tensor

from src.model.R2plus1D_18_MotionNet import R2plus1D_18_MotionNet # original model
# new models (small alterations)
from src.model.dropout_0_10_R2plus1D_18_MotionNet import dropout_0_10_R2plus1D_18_MotionNet 
from src.model.dropout_0_25_R2plus1D_18_MotionNet import dropout_0_25_R2plus1D_18_MotionNet 
from src.model.dropout_0_50_R2plus1D_18_MotionNet import dropout_0_50_R2plus1D_18_MotionNet 
from src.model.dropout_0_75_R2plus1D_18_MotionNet import dropout_0_75_R2plus1D_18_MotionNet 


from src.echonet_dataset import EchoNetDynamicDataset
from src.clasfv_losses import deformation_motion_loss, motion_seg_loss, DiceLoss, categorical_dice
from src.train_test import train, test



######
# for slider visualizations
%matplotlib widget
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import numpy as np

from scipy.ndimage import correlate
from skimage.filters import *

from ipywidgets import VBox, IntSlider, AppLayout
# initialize to use dark background ...
plt.style.use('dark_background')
######
# for creating gif animation and viewing them
from matplotlib import animation
from IPython.display import Image

######

import torch
import torch.nn as nn
import torch.optim as optim

import random
import pickle
import time

tic, toc = (time.time, time.time)

## Load in models

We have the luxury of trying inference on multiple models, since I'm trying to create slightly different models.

Original pre-trained model by Yida will be `model_name_1`.
The one that I will add dropout and k-fold cross validation to will be `model_name_2`.

In [ ]:
# hold tuples of (name, model object)
loaded_in_models = []

In [ ]:
for model_name in model_names:
    model_save_path = f"save_models/{model_name}"
    
    # original model
    if model_name == "Original-Pretrained-R2plus1DMotionSegNet_model.pth":
        # model = DDP(R2plus1D_18_MotionNet())
         model = torch.nn.DataParallel(R2plus1D_18_MotionNet())
        
    # altered models
    if model_name == "dropout-0_75-R2plus1DMotionSegNet_model.pth":
        # model = DDP(dropout_0_75_R2plus1D_18_MotionNet())
        model = torch.nn.DataParallel(dropout_0_75_R2plus1D_18_MotionNet())
    if model_name == "dropout-0_50-R2plus1DMotionSegNet_model.pth":
        # model = DDP(dropout_0_50_R2plus1D_18_MotionNet())
        model = torch.nn.DataParallel(dropout_0_50_R2plus1D_18_MotionNet())
    if model_name == "dropout-0_25-R2plus1DMotionSegNet_model.pth":
        # model = DDP(dropout_0_25_R2plus1D_18_MotionNet())
        model = torch.nn.DataParallel(dropout_0_25_R2plus1D_18_MotionNet())
    if model_name == "dropout-0_10-R2plus1DMotionSegNet_model.pth":
        # model = DDP(dropout_0_10_R2plus1D_18_MotionNet())
        model = torch.nn.DataParallel(dropout_0_10_R2plus1D_18_MotionNet())
    
    
    model.to("cuda")
    torch.cuda.empty_cache()
    model.load_state_dict(torch.load(model_save_path)["model"])
    print(f'{model_name} has {sum(p.numel() for p in model.parameters() if p.requires_grad)} parameters.')
    model.eval();
    
    loaded_in_models.append((model_name, model))

print(len(loaded_in_models))

## Load in Testing Dataset to do inference on pretrained model

In [ ]:
with open("fold_indexes/stanford_valid_sampled_indices", "rb") as infile:
    valid_mask = pickle.load(infile)
infile.close()

test_dataset = EchoNetDynamicDataset(split='test', clip_length="full", raise_for_es_ed=False, period=1)

In [ ]:
print(len(test_dataset))

### Grab a video to look at, can be random or manually choose one of the 1276 from test dataset
### For sake of comparison, let's look at first sample from the test dataset

In [ ]:
# test_pat_index = np.random.randint(len(test_dataset))
test_pat_index = 0 

video, (filename, EF, es_clip_index, ed_clip_index, es_index, ed_index, es_frame, ed_frame, es_label, ed_label) = test_dataset[test_pat_index]

In [ ]:
def divide_to_consecutive_clips(video, clip_length=32, interpolate_last=False):
    source_video = video.copy()
    video_length = video.shape[1]
    left = video_length % clip_length
    if left != 0 and interpolate_last:
        source_video = torch.Tensor(source_video).unsqueeze(0)
        source_video = F.interpolate(source_video, size=(int(np.round(video_length / clip_length) * clip_length), 112, 112),
                                     mode="trilinear", align_corners=False)
        source_video = source_video.squeeze(0).squeeze(0)
        source_video = source_video.numpy()
    
    videos = np.empty(shape=(1, 3, clip_length, 112, 112))

    for start in range(0, int(clip_length * np.round(video_length / clip_length)), clip_length):
        one_clip = source_video[:, start: start + clip_length]
        one_clip = np.expand_dims(one_clip, 0)
        videos = np.concatenate([videos, one_clip])
    return videos[1:]


# goes thru a video and annotates where we can start clips given video length, clip length, etc.
def get_all_possible_start_points(ed_index, es_index, video_length, clip_length):
    assert es_index - ed_index > 0, "not a ED to ES clip pair"
    possible_shift = clip_length - (es_index - ed_index)
    allowed_right = video_length - es_index
    if allowed_right < possible_shift:
        return np.arange(ed_index - possible_shift + 1, video_length - clip_length + 1)
    if possible_shift < 0:
        return np.array([ed_index])
    elif ed_index < possible_shift:
        return np.arange(ed_index + 1)
    else:
        return np.arange(ed_index - possible_shift + 1, ed_index + 1)

In [ ]:
possible_starts = get_all_possible_start_points(ed_index, es_index, video.shape[1], clip_length=32)
print(len(possible_starts))
print(possible_starts)

In [ ]:
ed_index

In [ ]:
es_index

### Segment All 32-Frame Clips

In [ ]:
# segment using all models
all_segmentation_outputs = []
all_motion_outputs = []

# for each model, segment the clips
for name, model in loaded_in_models:
    
    segmentation_outputs = np.empty(shape=(1, 2, 32, 112, 112))
    motion_outputs = np.empty(shape=(1, 4, 32, 112, 112))
    for start in possible_starts:
        one_clip = np.expand_dims(video[:, start: start + 32], 0)
        segmentation_output, motion_output = model(torch.Tensor(one_clip))
        segmentation_outputs = np.concatenate([segmentation_outputs, segmentation_output.cpu().detach().numpy()])
        motion_outputs = np.concatenate([motion_outputs, motion_output.cpu().detach().numpy()])
    segmentation_outputs = segmentation_outputs[1:]
    motion_outputs = motion_outputs[1:]
    
    # save 
    all_segmentation_outputs.append(segmentation_outputs)
    all_motion_outputs.append(motion_outputs)


In [ ]:
print(len(all_segmentation_outputs), len(all_motion_outputs))
print(len(all_motion_outputs[0]))

In [ ]:
all_motion_outputs[0].shape

## Define the function to create our Motion Output on ColorMap GIF Animations

How to save animation as gif: http://louistiao.me/posts/notebooks/save-matplotlib-animations-as-gifs/

In [ ]:
def create_32_frame_motion_colormap_gif(model_name, motion_output_obj, clip_index, which_direction, out_file_comment="", cmap="viridis"):
    # plot all 4, forward x,y and backward x, y
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15,5));
    
    # put a big title with model's file name
    plt.suptitle(f'{model_name} | Forwards forward in time | Backwards backward in time');
    
    start_frame = 0
    
    # find the max and min value needed for colormapping for all 4 motion outputs across ALL 
    # 32 frames
    all_mins = []
    all_maxes = []
    for frame_ind in range(32):
        for direction_ind in range(4):
            all_mins.append(motion_output_obj[clip_index][which_direction[direction_ind]][frame_ind].min())
            all_maxes.append(motion_output_obj[clip_index][which_direction[direction_ind]][frame_ind].max())
    
    color_min = min(all_mins)
    color_max = max(all_maxes)
    
    # for backward motions, we're going to go backwards as it that makes more sense. we will sync up the backward x,y 
    # they will go from frame 31 -> 0 while forward goes from 0 -> 31
    
    ax1.set_title("Forward X");
    ax1.set_xlabel('Width Pixel Index') # only display axes labels on first fig for clarity
    ax1.set_ylabel('Height Pixel Index')
    fx = motion_output_obj[clip_index][which_direction[0]][start_frame]
    ax1_img = ax1.imshow(fx, cmap=cmap, vmin=color_min, vmax=color_max);

    ax2.set_title("Forward Y")
    fy = motion_output_obj[clip_index][which_direction[1]][start_frame]
    ax2_img = ax2.imshow(fy, cmap=cmap, vmin=color_min, vmax=color_max);

    ax3.set_title("Backward X")
    bx = motion_output_obj[clip_index][which_direction[2]][31 - start_frame]
    ax3_img = ax3.imshow(bx, cmap=cmap, vmin=color_min, vmax=color_max);

    ax4.set_title("Backward Y")
    by = motion_output_obj[clip_index][which_direction[3]][31 - start_frame]
    ax4_img = ax4.imshow(by, cmap=cmap, vmin=color_min, vmax=color_max);
    # only need to show colorbar on the last figure, since colorbar is now the same across all figures.
    cbar_ax = fig.add_axes([0.92, 0.1, 0.01, 0.75])
    fig.colorbar(ax4_img, cax=cbar_ax)
    
    # funct to update imshow with new frame of motion output
    def animate(frame):
        fx = motion_output_obj[clip_index][which_direction[0]][frame]
        ax1_img = ax1.imshow(fx, cmap=cmap, vmin=color_min, vmax=color_max);
        # fig.colorbar(ax1_img, ax=ax1)
        fy = motion_output_obj[clip_index][which_direction[1]][frame]
        ax2_img = ax2.imshow(fy, cmap=cmap, vmin=color_min, vmax=color_max);
        # fig.colorbar(ax2_img, ax=ax2)

        bx = motion_output_obj[clip_index][which_direction[2]][31 - frame]
        ax3_img = ax3.imshow(bx, cmap=cmap, vmin=color_min, vmax=color_max);
        # fig.colorbar(ax3_img, ax=ax3)
        by = motion_output_obj[clip_index][which_direction[3]][31 - frame]
        ax4_img = ax4.imshow(by, cmap=cmap, vmin=color_min, vmax=color_max);
        # fig.colorbar(ax4_img, ax=ax4)

        return [ax1, ax2, ax3, ax4]
    anim = animation.FuncAnimation(fig, animate, np.arange(0, 32), interval=500, blit=True); # interval is milliseconds between each redraw, adjusts animation speed
    anim.save(f'./warren-random/visualization-outputs/{model_name}_motion_colormap_{out_file_comment}.gif', writer='imagemagick', fps=10);

## Make GIFs of motion outputs on a colormap from all the models that we have

In [ ]:
clip_index = -1   # last 32 frame clip
which_direction = [0,1,2,3] # in order of: forward x,y, backward x,y 

out_file_comment = "colorbar-seismic"
color_mapping = "viridis"

In [ ]:
for i in range(len(loaded_in_models)):
    create_32_frame_motion_colormap_gif(model_name = loaded_in_models[i][0],
                                    motion_output_obj = all_motion_outputs[i],
                                    clip_index = clip_index,
                                    which_direction = which_direction,
                                    out_file_comment = out_file_comment,
                                    cmap = color_mapping);

## Look at the Gifs:

In [ ]:
# model 1
Image(f'./warren-random/visualization-outputs/{loaded_in_models[0][0]}_motion_colormap_{out_file_comment}.gif')

In [ ]:
# model 2
Image(f'./warren-random/visualization-outputs/{loaded_in_models[1][0]}_motion_colormap_{out_file_comment}.gif')

In [ ]:
# model 3
Image(f'./warren-random/visualization-outputs/{loaded_in_models[2][0]}_motion_colormap_{out_file_comment}.gif')

In [ ]:
# model 4
Image(f'./warren-random/visualization-outputs/{loaded_in_models[3][0]}_motion_colormap_{out_file_comment}.gif')

In [ ]:
# model 5
Image(f'./warren-random/visualization-outputs/{loaded_in_models[4][0]}_motion_colormap_{out_file_comment}.gif')

## Create Vector Fields of our Motion Outputs

In [ ]:
def create_32_frame_motion_vector_field_gifs(model_name, motion_output_obj, clip_index, which_direction, out_file_comment=""):
    # 2 subplots, forward and backward motion
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10));
    
    # starting frame ind
    start_frame = 0
    
    # get initial forward and backward motion
    fx = motion_output_obj[clip_index][which_direction[0]][start_frame]
    fy = motion_output_obj[clip_index][which_direction[1]][start_frame]
    
    bx = motion_output_obj[clip_index][which_direction[2]][31 - start_frame]
    by = motion_output_obj[clip_index][which_direction[3]][31 - start_frame]
    
    # create values for the tails of the vectors (these don't need to change)
    nrows, ncols = fx.shape
    x_tmp = np.linspace(0, 112, ncols)  
    y_tmp = np.linspace(0, 112, nrows)
    x_tails, y_tails = np.meshgrid(x_tmp, y_tmp, indexing='xy')
    
    # put titles, axes on subplots
    plt.suptitle(f"Motion Vector Field (x,y) for {model_name} | Forwards forward in time | Backwards backward in time")
    
    ax1.set_title('Forward Motion')
    ax1.set_xlabel('$\Delta x$')
    ax1.set_ylabel('$\Delta y$')
    ax1.invert_yaxis()
    
    ax2.set_title('Backward Motion')
    ax2.set_xlabel('$\Delta x$')
    ax2.set_ylabel('$\Delta y$')
    ax2.invert_yaxis()
    
    # put initial magnitudes to the vectors with fixed tails
    # forward
    ax1.quiver(x_tails, y_tails, fx, fy, edgecolor='b', facecolor='none', linewidth=0.7);
    # backward
    ax2.quiver(x_tails, y_tails, bx, by, edgecolor='b', facecolor='none', linewidth=0.7);

    
    # funct to update imshow with new frame of motion output
    def animate(frame):
        # update forward xy and backward xy
        fx = motion_output_obj[clip_index][which_direction[0]][frame]
        fy = motion_output_obj[clip_index][which_direction[1]][frame]

        bx = motion_output_obj[clip_index][which_direction[2]][31 - frame]
        by = motion_output_obj[clip_index][which_direction[3]][31 - frame]
        
        # clear forward and backward axes first (we'll need to reapply the labels and inverting the y axis)
        ax1.clear()
        ax2.clear()
        ax1.set_title('Forward Motion')
        ax1.set_xlabel('$\Delta x$')
        ax1.set_ylabel('$\Delta y$')
        ax1.invert_yaxis()

        ax2.set_title('Backward Motion')
        ax2.set_xlabel('$\Delta x$')
        ax2.set_ylabel('$\Delta y$')
        ax2.invert_yaxis()
        
        # update the magnitudes of the vectors with fixed tails
        # forward
        ax1.quiver(x_tails, y_tails, fx, fy, edgecolor='b', facecolor='none', linewidth=0.7);
        # backward
        ax2.quiver(x_tails, y_tails, bx, by, edgecolor='b', facecolor='none', linewidth=0.7);

        
    anim = animation.FuncAnimation(fig, animate, np.arange(0, 32), interval=500, blit=True); # interval is milliseconds between each redraw, adjusts animation speed
    anim.save(f'./warren-random/visualization-outputs/{model_name}_motion_vector_field_{out_file_comment}.gif', writer='imagemagick', fps=10);

In [ ]:
clip_index = -1   # last 32 frame clip
which_direction = [0,1,2,3] # in order of: forward x,y, backward x,y 

out_file_comment = ""

In [ ]:
for i in range(len(loaded_in_models)):
    create_32_frame_motion_vector_field_gifs(model_name = loaded_in_models[i][0], 
                                             motion_output_obj = all_motion_outputs[i], 
                                             clip_index = clip_index, 
                                             which_direction = which_direction, 
                                             out_file_comment= out_file_comment);

## View Vector Fields

In [ ]:
# model 1
Image(f'./warren-random/visualization-outputs/{loaded_in_models[0][0]}_motion_vector_field_{out_file_comment}.gif')

In [ ]:
# model 2
Image(f'./warren-random/visualization-outputs/{loaded_in_models[1][0]}_motion_vector_field_{out_file_comment}.gif')

In [ ]:
# model 3
Image(f'./warren-random/visualization-outputs/{loaded_in_models[2][0]}_motion_vector_field_{out_file_comment}.gif')

In [ ]:
# model 4
Image(f'./warren-random/visualization-outputs/{loaded_in_models[3][0]}_motion_vector_field_{out_file_comment}.gif')

In [ ]:
# model 5 
Image(f'./warren-random/visualization-outputs/{loaded_in_models[4][0]}_motion_vector_field_{out_file_comment}.gif')

## Try to zoom in on the Vector Field Visualizations to get a better understanding of whether the vectors are in the right direction
## Also, use a slider instead, that way we can slowly compare.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button

# grab actual motion output information to try slider animation with.
# index represents which model in order of the loaded_in_models
ind = 0

motion_output_obj = all_motion_outputs[ind] 
model_name = loaded_in_models[ind][0]

clip_index = -1   # last 32 frame clip

which_direction = [0,1,2,3] # in order of: forward x,y, backward x,y 


###########

# plot initial vector field at t = 0

# 2 subplots, forward and backward motion
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10), sharex=True, sharey=True);


# starting frame ind
start_frame = 0

### this one, since we're sliding, we go forward in time for both forwards and backwards vector fields

# get initial forward and backward motion
fx = motion_output_obj[clip_index][which_direction[0]][start_frame]
fy = motion_output_obj[clip_index][which_direction[1]][start_frame]

bx = motion_output_obj[clip_index][which_direction[2]][start_frame]
by = motion_output_obj[clip_index][which_direction[3]][start_frame]

# create values for the tails of the vectors (these don't need to change)
nrows, ncols = fx.shape
x_tmp = np.linspace(0, 112, ncols)  
y_tmp = np.linspace(0, 112, nrows)
x_tails, y_tails = np.meshgrid(x_tmp, y_tmp, indexing='xy')

# put titles, axes on subplots
plt.suptitle(f"Motion Vector Field (x,y) for {model_name}")
ax1.set_title('Forward Motion')
ax1.set_xlabel('$\Delta x$')
ax1.set_ylabel('$\Delta y$')
ax1.invert_yaxis()

ax2.set_title('Backward Motion')
ax2.set_xlabel('$\Delta x$')
ax2.set_ylabel('$\Delta y$')
ax2.invert_yaxis()

# put initial magnitudes to the vectors with fixed tails
# forward
blah = ax1.quiver(x_tails, y_tails, fx, fy, edgecolor='b', facecolor='none', linewidth=0.7);
# backward
blah = ax2.quiver(x_tails, y_tails, bx, by, edgecolor='b', facecolor='none', linewidth=0.7);

######################

# update function
def update(frame):
    frame = int(frame)
    # update forward xy and backward xy
    fx = motion_output_obj[clip_index][which_direction[0]][frame]
    fy = motion_output_obj[clip_index][which_direction[1]][frame]

    bx = motion_output_obj[clip_index][which_direction[2]][frame]
    by = motion_output_obj[clip_index][which_direction[3]][frame]

    # clear forward and backward axes first (we'll need to reapply the labels and inverting the y axis)
    ax1.clear()
    ax2.clear()
    ax1.set_title('Forward Motion')
    ax1.set_xlabel('$\Delta x$')
    ax1.set_ylabel('$\Delta y$')
    ax1.invert_yaxis()

    ax2.set_title('Backward Motion')
    ax2.set_xlabel('$\Delta x$')
    ax2.set_ylabel('$\Delta y$')
    ax2.invert_yaxis()

    # update the magnitudes of the vectors with fixed tails
    # forward
    blah = ax1.quiver(x_tails, y_tails, fx, fy, edgecolor='b', facecolor='none', linewidth=0.7);
    # backward
    blah = ax2.quiver(x_tails, y_tails, bx, by, edgecolor='b', facecolor='none', linewidth=0.7);

##############
# connect slider to the update function

# Make a horizontal slider to control the frame we are looking at.
axframe = plt.axes([0.10, 0.1, 0.65, 0.03])

frame_slider = Slider(
    ax=axframe,
    label='Frame',
    valmin=0,
    valmax=31,
    valinit=0,
    valstep=1.0,
    valfmt=f"%0.f"
)

frame_slider.on_changed(update)

# Create a `matplotlib.widgets.Button` to reset the sliders to initial values.
resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', hovercolor='0.975')

def reset(event):
    frame_slider.reset()
    
button.on_clicked(reset)


plt.subplots_adjust(left=0.1, bottom=0.25)

# fig.show()

# plt.close()

## If our visualization doesn't tell us anything, we need to try to quantify how correct our motion outputs are. One of our losses that Yida uses to train the model should be telling us this information of how accurate our warps using the motion tracking information is relative to ground truth real frames. 

### How do we get the motion tracking information while it is training? We will have to step slowly thru the training cycle. we'll step thru a part of a single train cycle (epoch) on hopefully a single video from the train dataset , a couple of 32 frame clips from this single video to see what kind of motion tracking information we get and see how we can visualize the warping and comparisons that occur during the underlying training cycle (quantified by a loss)

### In order to visualizer a warping frame by frame, we need to calculate motion output for a specific frame and then apply it to the previous frame and see what it looks like. Will creating visualizations help at all, this feels like a useless endeavor?

1. Pass a single 32 frame clip from a test video to a loaded in model
2. Get its motion tracking information
3. Incrementally apply the warp using this motion tracking information and see if it matches the ground truth (whatever that's supposed to mean and look like)
4. Then we just stare at it, like with my other visualizations, and realize what ? I don't think I've gained any new insight.

